<a href="https://colab.research.google.com/github/turatig/frequent_itemsets/blob/master/frequent_itemsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MARKET BASKET ANALYSIS NOTEBOOK**

Dowload and preprocess dataset

In [1]:
!pip install kaggle


import os,sys,time,zipfile,json,re
import functools as ft
import itertools as it
from datetime import datetime as dt

os.environ['KAGGLE_USERNAME']='giacomoturati1'
os.environ['KAGGLE_KEY']='7d34a1aefc3558065164b70c24ce27ed'

from kaggle.api.kaggle_api_extended import KaggleApi

def get_dataset():

  #execute only if the dataset was not already downloaded
  if 'old-newspaper.tsv' not in os.listdir():
    api=KaggleApi()
    api.authenticate()

    api.dataset_download_file('alvations/old-newspapers','old-newspaper.tsv')

    with zipfile.ZipFile('old-newspaper.tsv.zip','r') as _zip:
      _zip.extractall()

#Yield baskets (list of lists) reading from tsv
#languages: subset of languages to be considered during the market-basket analysis
def iter_baskets_from_tsv(languages=None,max_basket=-1,skip=0):
  count=0
  with open('old-newspaper.tsv','r') as f:
    #skip header line
    next(f)
    for line in f:
      l=line.split('\t')
      if languages is not None and l[0] not in languages: continue

      #get a list of words as basket skipping any sequence of non-alphabetical characters
      basket=re.split(r'[^a-zA-Z]+',l[3])
      #remove any empty string
      basket=[word.lower() for word in basket if word!='']
      count+=1
      if count>skip: yield basket

      if max_basket>0 and count>=max_basket: break
      
    f.close()

#Create txt dataset. Structure: every line (basket) is a sequence of words (items) separated from commas
def create_txt_dataset(languages=None,max_basket=-1,skip=0):
  baskets=[]

  #execute only if the dataset was not already created
  for line in iter_baskets_from_tsv(languages,max_basket,skip=0):
    baskets.append(ft.reduce(lambda w1,w2: w1+','+w2,line))

  filename=ft.reduce(lambda i,j:i+'_'+j,languages).lower() if languages is not None else 'all_languages' 
  filename+=str(len(baskets))+'.txt'

  with open(filename,'w') as f:
    for basket in baskets:
      f.write(basket+'\n')

    f.close()

#Yield baskets from json structures as array of arrays
def iter_baskets_from_txt(filename,max_basket=-1,skip=0):
  theres_next=True
  basket=[]
  count=0

  with open(filename,'r') as f:
    eol=''
    for line in f:
      line=line.split(',')
      count+=1
      #trim \n
      if count>skip: yield line[:-1]+[line[-1][:-1]]
    
    f.close()

#Just to iterate agnostically over basket file
def iter_baskets(basket_file,max_basket=-1,skip=0):
  if callable(basket_file):
    for basket in basket_file(max_basket,skip): yield basket
  else:
    count=0
    for basket in basket_file:
      count+=1
      if count>skip: yield basket
      if max_basket>0 and count>=max_basket: break 

    


get_dataset()
create_txt_dataset(['Italian'],300)

Utilities to log the execution

In [2]:
def sizeof_GB(obj): return "%f"%(sys.getsizeof(obj)/1000000000)

#decorator to log time execution of function/method
def time_it(f):
  def _wrap(*args,**kwargs):
    start=time.time()
    res=f(*args,**kwargs)
    stop=time.time()
    
    print('\n'+'-'*30)
    print('Function {0} executed in {1} seconds'.format(f.__name__,stop-start))
    print('-'*30+'\n')
    return res
  return _wrap

#decorator to log the memory space used before and after a candidate itemset filtering operation
def log_filter(f):
  def _wrap(*args,**kwargs):
    if len(args)>0:
      print('Total number of candidate {0}-itemsets: {1}\nSize in GB: {2}'.\
            format(len(list(args[0].keys())[0]),len(args[0]),sizeof_GB(args[0])))
      
    #argument was given by key=value
    else:
      print('Total number of candidate {0}-itemsets: {1}\nSize in GB: {2}'.\
            format(len(list(kwargs['candidates'].keys())[0]),len(kwargs['candidates']),sizeof_GB(kwargs['candidates'])))
    
    res=f(*args,**kwargs)

    if res:
      print('Number of frequent {0}-itemsets: {1}\nSize in GB: {2}'.\
              format(len(list(res.keys())[0]),len(res),sizeof_GB(res)))
    else:
      print('Number of frequent {0}-itemsets: {1}\nSize in GB: {2}'.\
              format(0,0,0))
    print('-'*30+'\n')
    return res

  return _wrap

#Dump on json file the result of an algorithm run
def dump_result(algo,s,basket_count,freq_it_sets):
  def remap(dic):
    return {str(k):v for k,v in dic.items()}

  header_info={'support_threshold':s,'total_n_baskets':basket_count}

  filename=algo+'_market_basket_analysis_'+str(dt.today())[:10]+'_'+str(dt.today())[11:]+'.json'
              
  with open(filename,'w') as f:
    f.write(json.dumps([header_info]+[remap(dic) for dic in freq_it_sets],indent='\t'))
    f.close()

A-priori algorithm implementation

In [3]:
"""
  Filter candidate set of itemsets according to suppport threshold 
"""
@log_filter
@time_it
def filter_ck(candidates,s):
  return {k:v for k,v in candidates.items() if v>=s}

"""
  Discard unfrequent singletons from a basket
"""
def freq_sing(freq_it_sets,basket):
  return [word for word in basket if (word,) in freq_it_sets[1]]

"""
  Check monotonicity property
  kuple is a possible k-itemset -> all immediate subsets (k-1 itemsets) are frequent itemsets.
""" 
def check_mono_prop(kuple,k,freq_it_sets):
  return all([tuple(sorted(el)) in freq_it_sets[k-1] for el in it.combinations(kuple,r=k-1)])

"""
  Utility to clean output of analysis
"""
def clean_output(freq_it_sets):
  if not freq_it_sets[-1]:
    #remove the last element if empty
    freq_it_sets=freq_it_sets[:-1]
  #remove empty itemset set
  freq_it_sets=freq_it_sets[1:]
  return freq_it_sets

"""
  Return candidate k-itemsets found after a basket_file pass
"""
@time_it  
def apr_get_ck(basket_file,k,freq_it_sets,max_basket=-1,skip=0):
  basket_count=0
  candidates=dict()

  for basket in iter_baskets(basket_file,max_basket,skip):
    basket_count+=1
    if k>2:
      basket=freq_sing(freq_it_sets,basket)
              
    for kuple in it.combinations(basket,r=k):
        #sort tuple in order to avoid duplication of the same itemset considered in different order
        kuple=tuple(sorted(kuple))

        if check_mono_prop(kuple,k,freq_it_sets):
          if kuple not in candidates.keys(): candidates[kuple]=1
          else: candidates[kuple]+=1

  return candidates,basket_count

"""
  Apriori algorithm iteration
  freq_it_sets:the k-1-th element is the set of frequent itemsets made of k elements
"""
def apriori(basket_file,s=0,max_basket=-1,skip=0,max_k=-1,log=False,spark=False):
    freq_it_sets=[{tuple():1}]
    k=1

    #stop when no more frequent itemsets are found or k>max_k
    while freq_it_sets[-1] and (max_k<0 or k<=max_k):
      if spark:
        basket_file,next_basket=it.tee(basket_file,2)

      ck,basket_count=apr_get_ck(basket_file,k,freq_it_sets,max_basket,skip)
      if s<=0:
        #set threshold to the 1% of the total number of baskets
        s=basket_count//100
      freq_it_sets.append(filter_ck(ck,s))
      k+=1

      if spark:
        basket_file=next_basket

    freq_it_sets=clean_output(freq_it_sets)
    if log:dump_result("apriori",s,basket_count,freq_it_sets)

    return freq_it_sets
    

In [4]:
from functools import partial

basket_file=partial(iter_baskets_from_txt,'italian300.txt')
#res=apriori(basket_file,max_k=3)


PCY implementation

In [5]:
!pip install -q bitmap
from bitmap import BitMap

#map a tuple to a bucket of a table of size=s
def hash_tuple(t,s): return hash(t)%s
def set_all(bm):
  for i in bm.size():
    bm.set(i)


In [6]:
"""
  Return candidate k-itemsets found after a basket_file pass.
  bm: bitmap of frequent buckets of couples
"""
@time_it  
def pcy_get_ck(basket_file,k,freq_it_sets,bm,max_basket=-1,skip=0):
  basket_count=0
  candidates=dict()
  buckets=[0 for i in range(bm.size())]

  for basket in iter_baskets(basket_file,max_basket,skip):
    basket_count+=1
    if k>2:
      basket=freq_sing(freq_it_sets,basket)
              
    for kuple in it.combinations(basket,r=k):
        kuple=tuple(sorted(kuple))

        #PCY variant: added constraint for couple -> must hash to a frequent bucket
        if check_mono_prop(kuple,k,freq_it_sets) and (k!=2 or bm[hash_tuple(kuple,bm.size())]):
          if kuple not in candidates.keys(): candidates[kuple]=1
          else: candidates[kuple]+=1
        
    if k==1:
      #PCY variant: during the first pass hash couples to buckets
      for couple in it.combinations(basket,r=2):
          buckets[hash_tuple(tuple(sorted(couple)),bm.size())]+=1

  return candidates,basket_count,buckets

"""
  PCY algorithm iteration
"""
def pcy(basket_file,s=0,bm_size=256,max_basket=-1,skip=0,max_k=-1,log=False):
    freq_it_sets=[{tuple():1}]
    bm=BitMap(bm_size)
    k=1

    while freq_it_sets[-1] and (max_k<0 or k<=max_k):
      ck,basket_count,buckets=pcy_get_ck(basket_file,k,freq_it_sets,bm,max_basket,skip)
      if s<=0: s=basket_count//100
      freq_it_sets.append(filter_ck(ck,s))

      if k==1:
        #PCY variant:set bit of frequent buckets in the bitmap
        for i in range(len(buckets)):
          if buckets[i]>=s: bm.set(i)
      k+=1
    
    freq_it_sets=clean_output(freq_it_sets)
    if log:dump_result("pcy",s,basket_count,freq_it_sets)
    
    return freq_it_sets


In [7]:
res=pcy(basket_file,max_k=3)


------------------------------
Function pcy_get_ck executed in 1.1520586013793945 seconds
------------------------------

Total number of candidate 1-itemsets: 6439
Size in GB: 0.000295

------------------------------
Function filter_ck executed in 0.0008265972137451172 seconds
------------------------------

Number of frequent 1-itemsets: 1140
Size in GB: 0.000037
------------------------------


------------------------------
Function pcy_get_ck executed in 3.6062963008880615 seconds
------------------------------

Total number of candidate 2-itemsets: 118747
Size in GB: 0.005243

------------------------------
Function filter_ck executed in 0.02234649658203125 seconds
------------------------------

Number of frequent 2-itemsets: 37906
Size in GB: 0.001311
------------------------------


------------------------------
Function pcy_get_ck executed in 66.12510085105896 seconds
------------------------------

Total number of candidate 3-itemsets: 713202
Size in GB: 0.041943

--------

Toivonen algorithm implementation

In [7]:
#For simplicity total number of baskets is known and support threshold too
def toivonen(basket_file,s,n_basket,n_part=4):
  p=1/n_part
  ps=0.8*p*s

  step
  for i in range(n_part):


  

Spark installation

In [7]:
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


SON algorithm implementation

In [8]:
import findspark

os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='spark-2.4.8-bin-hadoop2.7'

findspark.init()

from pyspark.sql import SparkSession
import pyspark


In [12]:
spark=SparkSession.builder.master("local[*]").getOrCreate()
sc=spark.sparkContext

basket_rdd=sc.textFile('italian300.txt')
support_t=3//basket_rdd.getNumPartitions()

res=basket_rdd.mapPartitions(lambda chunk: apriori(chunk,max_k=3,spark=True)).take(1)

KeyboardInterrupt: ignored